In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hsoub/L8/emails.csv',sep=';' )

In [ ]:
import nltk

from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords

from nltk.stem import PorterStemmer

import string

import re

nltk.download('punkt')

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def preprocess_text(text):


    tokens = word_tokenize(text.lower())


    tokens = [token for token in tokens if token not in string.punctuation]


    stop_words = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words]


    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    preprocessed_text = ' '.join(tokens)

    preprocessed_text = re.sub(r'http\S+|www\S+', '', preprocessed_text)

    preprocessed_text = re.sub(r'\d+', '', preprocessed_text)
    return preprocessed_text

In [ ]:
df['processed_Message']=df['Message'].apply(preprocess_text)
df

,Spam,Message,processed_Message
0,0,Please call me at 8,pleas call
1,1,Free money is available for you,free money avail
2,0,I study he studies they are students I studied...,studi studi student studi yesterday
3,1,I am working at office now to 9 evening,work offic even
4,0,U dun say so early hor... U c already then say...,u dun say earli hor ... u c alreadi say ...
...,...,...,...
5222,0,"It‘s reassuring, in this crazy world.",‘ reassur crazi world
5223,0,Oh... Okie lor...We go on sat...,oh ... oki lor ... go sat ...
5224,1,You are awarded a SiPix Digital Camera! call 0...,award sipix digit camera call landlin deliver...
5225,0,"Hey chief, can you give me a bell when you get...",hey chief give bell get need talk royal visit ...


In [ ]:
df[df['Spam'] == 1].count()

Spam                 675
Message              675
processed_Message    675
dtype: int64

In [ ]:
spam_df = df[df['Spam'] == 1]
non_spam_df = df[df['Spam'] == 0]

max_class_count = len(non_spam_df)

spam_resampled = spam_df.sample(n=max_class_count, replace=True, random_state=42)

balanced_df = pd.concat([non_spam_df, spam_resampled])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

#

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus=balanced_df['processed_Message']
max_features=100

tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
tfidf_vectors = tfidf_vectorizer.fit_transform(corpus)

In [ ]:
from sklearn.model_selection import train_test_split

X = tfidf_vectors
y = balanced_df['Spam']

X = X.toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
214/214 [==============================] - 1s 3ms/step - loss: 0.3721 - accuracy: 0.8877
Epoch 2/10
214/214 [==============================] - 1s 3ms/step - loss: 0.1581 - accuracy: 0.9398
Epoch 3/10
214/214 [==============================] - 1s 3ms/step - loss: 0.1345 - accuracy: 0.9530
Epoch 4/10
214/214 [==============================] - 1s 3ms/step - loss: 0.1187 - accuracy: 0.9593
Epoch 5/10
214/214 [==============================] - 1s 3ms/step - loss: 0.1067 - accuracy: 0.9634
Epoch 6/10
214/214 [==============================] - 1s 3ms/step - loss: 0.0959 - accuracy: 0.9684
Epoch 7/10
214/214 [==============================] - 1s 3ms/step - loss: 0.0879 - accuracy: 0.9701
Epoch 8/10
214/214 [==============================] - 1s 4ms/step - loss: 0.0814 - accuracy: 0.9731
Epoch 9/10
214/214 [==============================] - 0s 2ms/step - loss: 0.0769 - accuracy: 0.9750
Epoch 10/10
214/214 [==============================] - 0s 2ms/step - loss: 0.0734 - accuracy: 0.9755

In [ ]:
import numpy as np
y_pred_prob = model.predict(X_test)
y_pred = np.round(y_pred_prob)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy*100,2))

72/72 [==============================] - 0s 2ms/step
Accuracy: 96.49


In [ ]:
message="call to get free prize one million dollars"

processed_message=preprocess_text(message)
vector=tfidf_vectorizer.transform([processed_message])
vector_dense=vector.toarray()

y_pred_prob = model.predict(vector_dense)

y_pred = np.round(y_pred_prob)
if y_pred>0.5:
    print("Spam")
else:
    print("Not Spam")

1/1 [==============================] - 0s 45ms/step
Spam
